In [241]:
import numpy as np
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [242]:
doc1 = "Text Analytics is boring boring boring!!"
doc2 = "Analytics is interesting"
doc3 = "We want sports analytics"

In [243]:
def clean_txt(sent):
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english") + list(punctuation) 
    final_word = [term for term in tokens if term not in stop_updated 
               and len(term) > 2] 
    res = " ".join(final_word)
    return res

In [244]:
doc1_clean = clean_txt(doc1)
doc1_clean

'text analytics boring boring boring'

In [245]:
doc2_clean = clean_txt(doc2)
doc2_clean

'analytics interesting'

In [246]:
doc3_clean = clean_txt(doc3)
doc3_clean

'want sports analytics'

In [247]:
doc = pd.DataFrame([doc1_clean, doc2_clean, doc3_clean], columns=["text"])
doc

,text
0,text analytics boring boring boring
1,analytics interesting
2,want sports analytics


In [248]:
#CountVectorizer of sklearn gives us the DTM
from sklearn.feature_extraction.text import CountVectorizer

In [249]:
#Instantiating CountVectorizer
count_vect = CountVectorizer(ngram_range =(1,1))

In [250]:
#Invoking fit_transoform on the CountVectorizer object
X = count_vect.fit_transform(doc['text'])
X

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [251]:
X.toarray()

array([[1, 3, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1]], dtype=int64)

In [252]:
X.todense()

matrix([[1, 3, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 1]], dtype=int64)

- fit -> extract distinct words to form a bag of words (BoW
- transform -> compute the occourrence of every word from the BOW in each document

In [253]:
#Obtaining the BOW's
bow = count_vect.get_feature_names()
bow

['analytics', 'boring', 'interesting', 'sports', 'text', 'want']

In [254]:
#Get the position of every word in the BOW, as a dictionary
print(count_vect.vocabulary_)

{'text': 4, 'analytics': 0, 'boring': 1, 'interesting': 2, 'want': 5, 'sports': 3}


In [255]:
#To get the position of a particular word in BOW
count_vect.vocabulary_['boring']

1

In [256]:
#Printing the document term matrix can be done using toarray() or todense()
#methods
X.toarray()

array([[1, 3, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1]], dtype=int64)

In [257]:
X.todense()

matrix([[1, 3, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 1]], dtype=int64)

In [258]:
DTM = pd.DataFrame(X.toarray(),columns = count_vect.get_feature_names())
DTM

,analytics,boring,interesting,sports,text,want
0,1,3,0,0,1,0
1,1,0,1,0,0,0
2,1,0,0,1,0,1


In [259]:
DTM = pd.DataFrame(X.toarray(),columns = bow)
DTM

,analytics,boring,interesting,sports,text,want
0,1,3,0,0,1,0
1,1,0,1,0,0,0
2,1,0,0,1,0,1


In [260]:
#Term document matrix or TDM is a transpose of DTM 
#which is used in finding similarity between words
TDM = DTM.T
TDM

,0,1,2
analytics,1,1,1
boring,3,0,0
interesting,0,1,0
sports,0,0,1
text,1,0,0
want,0,0,1


In [261]:
ndoc = ["awesome paper is not always interesting","is paper really news"]

In [262]:
count_vect.transform(ndoc)

<2x6 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [263]:
DTMnd = pd.DataFrame(count_vect.transform(ndoc).toarray(),columns = count_vect.get_feature_names())
DTMnd

,analytics,boring,interesting,sports,text,want
0,0,0,1,0,0,0
1,0,0,0,0,0,0


## ngram analysis
- unigram -> a token comprises of a exactly a single word
- bigram -> a token comprises of exactly two words
- trigram -> a token comprises of eaxctly three words

### The next immediate word(s) are grouped with the current word to form a bigram or a tri-gram
- For example: suppose we have a string " how are you doing"
- unigram :{"how", "are", "you", "doing"}
- bigram: {"how are", "are you", "you doing"}
- trigram:{"how are you", "are you doing"}

### Any combination of unigrams, bigrams and trigrams can be obtained using a Countvectorizer

- By default the vectorizer's in sklearn will perform on unigrams, i.e ngram_range will be set to (1,1)
- to extarct bigrams ngram_range = (2,2)
- to extract trigrams ngram_range = (3,3)
- To extract both unigrams and bigrams ngram_range = (1,2)

In [264]:
# A token is called as a bigram if its min and max length is 2
count_vect_bg = CountVectorizer(ngram_range=(2,2))

In [265]:
X_bg = count_vect_bg.fit_transform(doc['text'])

In [266]:
doc['text']

0    text analytics boring boring boring
1                  analytics interesting
2                  want sports analytics
Name: text, dtype: object

In [267]:
print(count_vect_bg.get_feature_names())

['analytics boring', 'analytics interesting', 'boring boring', 'sports analytics', 'text analytics', 'want sports']


In [268]:
DTM_bg = pd.DataFrame(X_bg.toarray(),columns=count_vect_bg.get_feature_names())
DTM_bg

,analytics boring,analytics interesting,boring boring,sports analytics,text analytics,want sports
0,1,0,2,0,1,0
1,0,1,0,0,0,0
2,0,0,0,1,0,1


In [269]:
DTM_bg.sum()

analytics boring         1
analytics interesting    1
boring boring            2
sports analytics         1
text analytics           1
want sports              1
dtype: int64

In [270]:
count_vect_ubg = CountVectorizer(ngram_range=(1,1),max_features = 6)

In [271]:
X_ubg = count_vect_ubg.fit_transform(doc['text'])

In [272]:
print(count_vect_ubg.get_feature_names())

['analytics', 'boring', 'interesting', 'sports', 'text', 'want']


In [273]:
DTM_ubg = pd.DataFrame(X_ubg.toarray(),
                      columns=count_vect_ubg.get_feature_names())
DTM_ubg

,analytics,boring,interesting,sports,text,want
0,1,3,0,0,1,0
1,1,0,1,0,0,0
2,1,0,0,1,0,1


 ### Setting max features
 
 1. your vectorizer's take another argument called max_features where in you can specify the top n features to be selected
 2. on your DTM, if you perfrom column sum, you will get the total No. of occourrence of a word accross all the documents

In [274]:
#Get the top 3 features in our DTM based on their frequencies
#accross all the documents in the corpus by taking a column sum of all the words in the DTM
count_nfeatures = CountVectorizer(max_features=6,ngram_range=(1,1))

In [275]:
X_nfeatures = count_nfeatures.fit_transform(doc['text'])
doc['text']

0    text analytics boring boring boring
1                  analytics interesting
2                  want sports analytics
Name: text, dtype: object

In [276]:
count_nfeatures.get_feature_names()

['analytics', 'boring', 'interesting', 'sports', 'text', 'want']

In [277]:
DTM_nfeatures = pd.DataFrame(X_nfeatures.toarray(),
                            columns=count_nfeatures.get_feature_names())
DTM_nfeatures

,analytics,boring,interesting,sports,text,want
0,1,3,0,0,1,0
1,1,0,1,0,0,0
2,1,0,0,1,0,1


In [278]:
# Get the DTM for all the trigrams - CountVectorizer(ngram_range=(3,3))

### TFIDF

In [279]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [280]:
tfidf_vect = TfidfVectorizer(ngram_range=(1,1))

In [281]:
X = tfidf_vect.fit_transform(doc['text'])
X

<3x6 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [282]:
doc['text']

0    text analytics boring boring boring
1                  analytics interesting
2                  want sports analytics
Name: text, dtype: object

In [283]:
tfidf = pd.DataFrame(X.toarray(), columns=tfidf_vect.get_feature_names())
tfidf

,analytics,boring,interesting,sports,text,want
0,0.183595,0.932558,0.000000,0.000000,0.310853,0.000000
1,0.508542,0.000000,0.861037,0.000000,0.000000,0.000000
2,0.385372,0.000000,0.000000,0.652491,0.000000,0.652491


In [284]:
tfidf_square = tfidf.apply(lambda x: x**2)
tfidf_square

,analytics,boring,interesting,sports,text,want
0,0.033707,0.869664,0.000000,0.000000,0.096629,0.000000
1,0.258615,0.000000,0.741385,0.000000,0.000000,0.000000
2,0.148511,0.000000,0.000000,0.425744,0.000000,0.425744


In [285]:
doc['text']

0    text analytics boring boring boring
1                  analytics interesting
2                  want sports analytics
Name: text, dtype: object

In [286]:
tfidf_square.sum(axis=1)

0    1.0
1    1.0
2    1.0
dtype: float64

### Cosine Similarity

In [287]:
from sklearn.metrics.pairwise import cosine_similarity

In [288]:
cs = cosine_similarity(DTM)
print(cs)

[[1.         0.21320072 0.17407766]
 [0.21320072 1.         0.40824829]
 [0.17407766 0.40824829 1.        ]]


In [289]:
cs_words = cosine_similarity(DTM.T)

In [290]:
sim_mat = pd.DataFrame(cs_words,columns=DTM.columns,index=DTM.columns)
sim_mat

,analytics,boring,interesting,sports,text,want
analytics,1.00000,0.57735,0.57735,0.57735,0.57735,0.57735
boring,0.57735,1.00000,0.00000,0.00000,1.00000,0.00000
interesting,0.57735,0.00000,1.00000,0.00000,0.00000,0.00000
sports,0.57735,0.00000,0.00000,1.00000,0.00000,1.00000
text,0.57735,1.00000,0.00000,0.00000,1.00000,0.00000
want,0.57735,0.00000,0.00000,1.00000,0.00000,1.00000
